In [1]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
import random
import re
import string
import pickle
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import nltk
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.compose import ColumnTransformer
from category_encoders import TargetEncoder
from category_encoders.one_hot import OneHotEncoder


from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from catboost import CatBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score

# Example of connection with database

In [5]:
database = ''
user = ''
password = ''
host = ''
port = 0

In [6]:
CONNECTION = f"postgresql://{user}:{password}@{host}:{port}/{database}"

# Reading data

In [8]:
user_data = pd.read_sql(
    """ SELECT * FROM public.user_data;""",
    con=CONNECTION
)
user_data

,user_id,gender,age,country,city,exp_group,os,source
0,200,1,34,Russia,Degtyarsk,3,Android,ads
1,201,0,37,Russia,Abakan,0,Android,ads
2,202,1,17,Russia,Smolensk,4,Android,ads
3,203,0,18,Russia,Moscow,1,iOS,ads
4,204,0,36,Russia,Anzhero-Sudzhensk,3,Android,ads
...,...,...,...,...,...,...,...,...
163200,168548,0,36,Russia,Kaliningrad,4,Android,organic
163201,168549,0,18,Russia,Tula,2,Android,organic
163202,168550,1,41,Russia,Yekaterinburg,4,Android,organic
163203,168551,0,38,Russia,Moscow,3,iOS,organic


In [9]:
post_data = pd.read_sql(
    """ SELECT * FROM public.post_text_df;""",
    con=CONNECTION
)
post_data.head(10)

,post_id,text,topic
0,1,UK economy facing major risks\n\nThe UK manufa...,business
1,2,Aids and climate top Davos agenda\n\nClimate c...,business
2,3,Asian quake hits European shares\n\nShares in ...,business
3,4,India power shares jump on debut\n\nShares in ...,business
4,5,Lacroix label bought by US firm\n\nLuxury good...,business
5,6,Insurance bosses plead guilty\n\nAnother three...,business
6,14,Saudi investor picks up the Savoy\n\nLondons f...,business
7,2366,RACV | Annual cost of owning and maintaining a...,covid
8,7,Turkey-Iran mobile deal at risk\n\nTurkeys inv...,business
9,8,"Parmalat to return to stockmarket\n\nParmalat,...",business


In [10]:
# Base consists of 76.892.800 notes, but using only 2.000.000
whole_data = pd.read_sql(
    """ SELECT * 
        FROM public.feed_data as fa 
        LIMIT 2000000
        """,
    con=CONNECTION
)

whole_data

,timestamp,user_id,post_id,action,target
0,2021-12-27 18:48:47,151817,905,view,0
1,2021-12-27 18:51:04,151817,6936,view,0
2,2021-12-27 18:51:43,151817,5096,view,0
3,2021-12-27 18:54:23,151817,6497,view,0
4,2021-12-27 18:55:42,151817,3542,view,0
...,...,...,...,...,...
1999995,2021-11-22 16:03:41,111435,3967,view,0
1999996,2021-11-22 16:05:31,111435,1812,view,0
1999997,2021-11-22 16:07:02,111435,3429,view,0
1999998,2021-11-23 17:46:15,111435,433,view,0


# Data processing

## post_data

In [11]:
lematizer = WordNetLemmatizer()

def get_wordnet_pos(word):
    """Map part-of-speech tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

def del_letter_next_number(line):
    """delete letters next to numbers and numbers themselves"""
    list_inds_del = []
    skip_inds = 0
    len_line = len(line)
    list_nums = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
    for ind in range(len_line):
        if skip_inds != 0:
            skip_inds -= 1
            continue
        if line[ind] in list_nums:
            len_syms = 0
            len_nums = 0
            while (ind - 1 - len_syms >= 0) and (line[ind - 1 - len_syms] != ' '):
                len_syms += 1
            if (ind + 1 < len_line) and line[ind + 1] != ' ':
                len_nums += 1
                while (ind + len_nums < len_line) and line[ind + len_nums] != ' ':
                    len_nums += 1
            if len_nums == 0:
                list_inds_del.append((ind - len_syms, ind + 1))
            else:
                list_inds_del.append((ind - len_syms, ind + len_nums))
            skip_inds = len_nums          
    diff_i_j = 0
    for i, j in list_inds_del:
        line = line[:i - diff_i_j] + line[j - diff_i_j:]
        diff_i_j += j - i
    return line

def preprocessing(line, token=lematizer):
    """delete punctuation and apply WordNetLemmatizer()"""
    line = line.lower()
    line = line.replace('\n\n', ' ').replace('\n', ' ')
    line = del_letter_next_number(line)
    line = re.sub(r"[{}]".format(string.punctuation), " ", line)
    line = ' '.join([token.lemmatize(w, get_wordnet_pos(w)) for w in nltk.word_tokenize(line)])
    return line

In [12]:
def tf_idf():
    """Carrying out tf_idf method"""

    tfidf = TfidfVectorizer(
        stop_words='english',
        preprocessor=preprocessing
    )
    
    tfidf_data = (
        tfidf
        .fit_transform(post_data['text'])
        .toarray()
    )
    
    tfidf_data = pd.DataFrame(
        tfidf_data,
        index=post_data.post_id,
        columns=tfidf.get_feature_names_out()
    )
    
    return tfidf_data

In [19]:
tfidf_transform = tf_idf()

post_data['TotalTfIdf'] = tfidf_transform.sum(axis=1).reset_index()[0]
post_data['MaxTfIdf'] = tfidf_transform.max(axis=1).reset_index()[0]
post_data['MeanTfIdf'] = tfidf_transform.mean(axis=1).reset_index()[0]

post_data.head()

,post_id,text,topic,TotalTfIdf,MaxTfIdf,MeanTfIdf
0,1,UK economy facing major risks\n\nThe UK manufa...,business,8.189184,0.508965,0.000214
1,2,Aids and climate top Davos agenda\n\nClimate c...,business,11.169689,0.322813,0.000292
2,3,Asian quake hits European shares\n\nShares in ...,business,11.939820,0.262184,0.000312
3,4,India power shares jump on debut\n\nShares in ...,business,6.138186,0.559407,0.000160
4,5,Lacroix label bought by US firm\n\nLuxury good...,business,6.019855,0.432699,0.000157


In [20]:
def post_clustering(n_components, n_cluster):
    """At first, carrying out PCA with n_components, thereafter KMeans with n_cluster"""
    centered = tfidf_transform - tfidf_transform.mean()

    pca = PCA(n_components=n_components)
    pca_transform = pca.fit_transform(centered)


    kmeans = KMeans(n_clusters=n_cluster, random_state=0).fit(pca_transform)
    post_data['TextCluster'] = kmeans.labels_

    dists_cols = [f"DistanceTo{ith}thCluster" for ith in range(1, n_cluster+1)]

    return pd.DataFrame(
        data=kmeans.transform(pca_transform),
        columns=dists_cols
    )

In [21]:
post_data = pd.concat((post_data, post_clustering(n_components=20, n_cluster=12)), axis=1)

post_data.head()

,post_id,text,topic,TotalTfIdf,MaxTfIdf,MeanTfIdf,TextCluster,DistanceTo1thCluster,DistanceTo2thCluster,DistanceTo3thCluster,DistanceTo4thCluster,DistanceTo5thCluster,DistanceTo6thCluster,DistanceTo7thCluster,DistanceTo8thCluster,DistanceTo9thCluster,DistanceTo10thCluster,DistanceTo11thCluster,DistanceTo12thCluster
0,1,UK economy facing major risks\n\nThe UK manufa...,business,8.189184,0.508965,0.000214,5,0.492331,0.447937,0.435919,0.559333,0.450724,0.146886,0.538823,0.512480,0.401425,0.456120,0.486820,0.544076
1,2,Aids and climate top Davos agenda\n\nClimate c...,business,11.169689,0.322813,0.000292,8,0.385516,0.304175,0.286827,0.456550,0.303240,0.295194,0.285362,0.387628,0.187096,0.315004,0.351551,0.448366
2,3,Asian quake hits European shares\n\nShares in ...,business,11.939820,0.262184,0.000312,5,0.435400,0.332740,0.318034,0.473531,0.336893,0.136727,0.496205,0.407744,0.257286,0.351483,0.377735,0.483131
3,4,India power shares jump on debut\n\nShares in ...,business,6.138186,0.559407,0.000160,8,0.391246,0.297098,0.264698,0.405269,0.287612,0.258797,0.448462,0.383844,0.182031,0.322498,0.348413,0.451525
4,5,Lacroix label bought by US firm\n\nLuxury good...,business,6.019855,0.432699,0.000157,8,0.301450,0.175777,0.140786,0.381345,0.232830,0.300775,0.391249,0.300300,0.078862,0.235685,0.260410,0.385714


## whole_data

In [22]:
"""delete data where an action is like because it is doubling"""
whole_data = whole_data[whole_data['action'] != 'like']

In [23]:
whole_data['target'].value_counts()

0    1576387
1     211804
Name: target, dtype: int64

# Everything merges into whole_data

In [24]:
whole_data = pd.merge(
    whole_data,
    post_data,
    on='post_id',
    how='left'
)

whole_data = pd.merge(
    whole_data,
    user_data,
    on='user_id',
    how='left'
)

whole_data.head()

,timestamp,user_id,post_id,action,target,text,topic,TotalTfIdf,MaxTfIdf,MeanTfIdf,...,DistanceTo10thCluster,DistanceTo11thCluster,DistanceTo12thCluster,gender,age,country,city,exp_group,os,source
0,2021-12-27 18:48:47,151817,905,view,0,Wine comedy wins critics award\n\nQuirky comed...,entertainment,8.134262,0.352992,0.000212,...,0.614519,0.567958,0.255163,1,28,Finland,Espoo,0,Android,organic
1,2021-12-27 18:51:04,151817,6936,view,0,I have enjoyed Criminal Intent series of Law a...,movie,6.395410,0.428570,0.000167,...,0.318168,0.284837,0.415374,1,28,Finland,Espoo,0,Android,organic
2,2021-12-27 18:51:43,151817,5096,view,0,"Stephen Hawking has one of the greatest minds,...",movie,9.753854,0.417807,0.000255,...,0.221069,0.184141,0.383842,1,28,Finland,Espoo,0,Android,organic
3,2021-12-27 18:54:23,151817,6497,view,0,Jack Black is an annoying character.This is an...,movie,5.872793,0.478013,0.000153,...,0.253580,0.194389,0.382218,1,28,Finland,Espoo,0,Android,organic
4,2021-12-27 18:55:42,151817,3542,view,0,If Herman Cain had died of COVID after a Biden...,covid,2.857774,0.657694,0.000075,...,0.253125,0.267927,0.392759,1,28,Finland,Espoo,0,Android,organic


In [25]:
whole_data['hour'] = whole_data['timestamp'].apply(lambda x: x.hour)
whole_data['dayofweek'] = whole_data['timestamp'].apply(lambda x: x.weekday())
whole_data['month'] = whole_data['timestamp'].apply(lambda x: x.month)

whole_data = whole_data.drop(['action','text'], axis=1)

whole_data = whole_data.set_index(['user_id', 'post_id'])

whole_data.head()

timestamp  target          topic  TotalTfIdf  \
user_id post_id                                                          
151817  905     2021-12-27 18:48:47       0  entertainment    8.134262   
        6936    2021-12-27 18:51:04       0          movie    6.395410   
        5096    2021-12-27 18:51:43       0          movie    9.753854   
        6497    2021-12-27 18:54:23       0          movie    5.872793   
        3542    2021-12-27 18:55:42       0          covid    2.857774   

                 MaxTfIdf  MeanTfIdf  TextCluster  DistanceTo1thCluster  \
user_id post_id                                                           
151817  905      0.352992   0.000212           11              0.660440   
        6936     0.428570   0.000167            7              0.359963   
        5096     0.417807   0.000255            1              0.307497   
        6497     0.478013   0.000153            1              0.307474   
        3542     0.657694   0.000075            2              0.260662   

                 DistanceTo2thCluster  DistanceTo3thCluster  ...  gender  age  \
user_id post_id                                              ...                
151817  905                  0.584698              0.605730  ...       1   28   
        6936                 0.194370              0.254921  ...       1   28   
        5096                 0.067425              0.158264  ...       1   28   
        6497                 0.092587              0.165230  ...       1   28   
        3542                 0.170079              0.065903  ...       1   28   

                 country   city  exp_group       os   source  hour  dayofweek  \
user_id post_id                                                                 
151817  905      Finland  Espoo          0  Android  organic    18          0   
        6936     Finland  Espoo          0  Android  organic    18          0   
        5096     Finland  Espoo          0  Android  organic    18          0   
        6497     Finland  Espoo          0  Android  organic    18          0   
        3542     Finland  Espoo          0  Android  organic    18          0   

                 month  
user_id post_id         
151817  905         12  
        6936        12  
        5096        12  
        6497        12  
        3542        12  

[5 rows x 29 columns]

# Train-test split

In [26]:
min(whole_data.timestamp), max(whole_data.timestamp)

(Timestamp('2021-10-01 06:02:14'), Timestamp('2021-12-29 23:44:39'))

In [27]:
train = whole_data[whole_data.timestamp < '2021-12-16']
test = whole_data[whole_data.timestamp >= '2021-12-16']

whole_data = whole_data.drop('timestamp', axis=1)
train = train.drop('timestamp', axis=1)
test = test.drop('timestamp', axis=1)

X_train = train.drop('target', axis=1)
X_test = test.drop('target', axis=1)

y_train = train['target']
y_test = test['target']

# Realization of ColumnTransfomer

In [28]:
def data_processing(whole_data, categorical_cols):
    one_hot_cols = [col for col in categorical_cols if whole_data[col].nunique() <= 5]
    target_cols = [col for col in categorical_cols if whole_data[col].nunique() > 5]
    
    one_hot_cols_inds = [list(X_train.columns).index(col) for col in one_hot_cols]
    target_cols_inds = [list(X_train.columns).index(col) for col in target_cols]
    
    t = [
        ("OneHotEncoder", OneHotEncoder(), one_hot_cols_inds),
        ("TargetEncoder", TargetEncoder(), target_cols_inds)
    ]
    
    return ColumnTransformer(transformers=t)

In [29]:
categorical_cols = ['gender', 'TextCluster', 'country', 'city', 'exp_group', 'os', 'source', 'topic', 'hour', 'dayofweek', 'month']
transformer = data_processing(whole_data, categorical_cols)

# Training

In [30]:
def train(name, model, param_grid, categorical_cols=None):
    search = GridSearchCV(model, param_grid, scoring='roc_auc')
    
    if categorical_cols:
        search.fit(X_train, y_train, cat_features=categorical_cols)
    else:
        search.fit(X_train, y_train)

    best_model = search.best_estimator_

    predict_prob_train = best_model.predict_proba(X_train)[:, 1]
    predict_prob_test = best_model.predict_proba(X_test)[:, 1]

    print(f"---{name}---")
    print(f"roc-auc score on train data: {roc_auc_score(y_train, predict_prob_train)}")
    print(f"roc-auc score on test data: {roc_auc_score(y_test, predict_prob_test)}")
    return best_model

### Models

In [31]:
logreg = Pipeline([("ColumnTransformer", transformer),
                      ("LR", LogisticRegression(random_state=21, class_weight='balanced'))])
param_grid_1 = {
    "LR__C": [0.01, 0.1, 1, 5, 10],
}

catboost = CatBoostClassifier(random_state=21, verbose=0)

param_grid_2 = {
    'iterations': [100, 150],
    'depth': [2, 4],
    'l2_leaf_reg': [5, 10],
}

param_grid = [param_grid_1, param_grid_2]
models = [logreg, catboost]
name = ['LogisticRegression', 'CatBoostClassifier']
best_models = []

for i in range(len(models)):
    if i == 0:
        best_model = train(name[0], models[0], param_grid[0])
        print()
    else:
        best_model = train(name[1], models[1], param_grid[1], categorical_cols)
    best_models.append(best_model)

---LogisticRegression---
roc-auc score on train data: 0.6395372325660763
roc-auc score on test data: 0.6179015358295846

---CatBoostClassifier---
roc-auc score on train data: 0.6476888733696118
roc-auc score on test data: 0.6330765016771561


# Save two models for A/B testing

In [32]:
pickle.dump(best_models[0], open('model_control.pkl', 'wb'))
pickle.dump(best_models[1], open('model_test.pkl', 'wb'))

# Download post_data in base

In [36]:
post_data.to_sql(    
   "post_processed_features",                    
    con=CONNECTION,
    schema="public", 
    if_exists='replace',
    index=False
   )      

23

In [37]:
test_ = pd.read_sql(
    """SELECT * FROM public.post_processed_features""",
    con=CONNECTION
)

test_

,post_id,text,topic,TotalTfIdf,MaxTfIdf,MeanTfIdf,TextCluster,DistanceTo1thCluster,DistanceTo2thCluster,DistanceTo3thCluster,DistanceTo4thCluster,DistanceTo5thCluster,DistanceTo6thCluster,DistanceTo7thCluster,DistanceTo8thCluster,DistanceTo9thCluster,DistanceTo10thCluster,DistanceTo11thCluster,DistanceTo12thCluster
0,1,UK economy facing major risks\n\nThe UK manufa...,business,8.189184,0.508965,0.000214,5,0.492331,0.447937,0.435919,0.559333,0.450724,0.146886,0.538823,0.512480,0.401425,0.456120,0.486820,0.544076
1,2,Aids and climate top Davos agenda\n\nClimate c...,business,11.169689,0.322813,0.000292,8,0.385516,0.304175,0.286827,0.456550,0.303240,0.295194,0.285362,0.387628,0.187096,0.315004,0.351551,0.448366
2,3,Asian quake hits European shares\n\nShares in ...,business,11.939820,0.262184,0.000312,5,0.435400,0.332740,0.318034,0.473531,0.336893,0.136727,0.496205,0.407744,0.257286,0.351483,0.377735,0.483131
3,4,India power shares jump on debut\n\nShares in ...,business,6.138186,0.559407,0.000160,8,0.391246,0.297098,0.264698,0.405269,0.287612,0.258797,0.448462,0.383844,0.182031,0.322498,0.348413,0.451525
4,5,Lacroix label bought by US firm\n\nLuxury good...,business,6.019855,0.432699,0.000157,8,0.301450,0.175777,0.140786,0.381345,0.232830,0.300775,0.391249,0.300300,0.078862,0.235685,0.260410,0.385714
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7018,7315,"OK, I would not normally watch a Farrelly brot...",movie,6.172745,0.358916,0.000161,7,0.424376,0.250755,0.347268,0.506282,0.415990,0.466603,0.523427,0.149893,0.370610,0.384682,0.331634,0.496161
7019,7316,I give this movie 2 stars purely because of it...,movie,6.196249,0.416536,0.000162,7,0.381782,0.201868,0.271204,0.461630,0.336536,0.410290,0.479172,0.080942,0.296417,0.337861,0.245771,0.422664
7020,7317,I cant believe this film was allowed to be mad...,movie,5.361235,0.593787,0.000140,10,0.328287,0.163004,0.228261,0.432512,0.328993,0.398501,0.467207,0.286087,0.267812,0.302624,0.070360,0.388009
7021,7318,The version I saw of this film was the Blockbu...,movie,6.794270,0.240499,0.000177,10,0.334147,0.126583,0.196120,0.413871,0.302178,0.362250,0.438816,0.267935,0.231110,0.271523,0.096919,0.324176
